<a href="https://colab.research.google.com/github/guarnieri0/GAIN/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import librerie**

In [163]:
import pandas as pd 

import json
import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy.linalg import norm
from numpy import savetxt
from sklearn import preprocessing
from openpyxl import Workbook
from openpyxl import load_workbook


**Reading Dataset**

In [164]:
df = pd.read_json("dataset.json")

**Deleting useless columns**

In [165]:
df = df.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)


**Retrieving the Error Code and Error description**

In [166]:
df_splitted= df['compareInfo'].str.split(pat=',', expand=True)
df_splitted = df_splitted.drop([0,1], axis = 1)
error_code = df_splitted.iloc[:,0].to_list()
error_description = df_splitted.iloc[:,1].to_list()
df.insert(5, 'error_description', error_description)
df.insert(6,'error_code',error_code)



**Retrieving the test category**

In [167]:
testname_splitted = df['testName'].str.split(pat='\\', expand = True)
testname_splitted = testname_splitted.drop([0,1,2,4,5], axis = 1)
testname = testname_splitted.iloc[:,0].to_list()
df.insert(7,'cat_name', testname)



**Deleting redundant columns**

In [168]:
df = df.drop(['compareInfo','reportTitle'], axis = 1)

**Filling the NULL value**

---

In [169]:
df['error_description'] = df['error_description'].fillna('None')
df['error_code'] = df['error_code'].fillna('None')


**Encoding text features**

In [170]:
encoder_df_error_code = LabelEncoder()
encoder_df_error_description = LabelEncoder()
encoder_df_cat_name = LabelEncoder()
df['error_code'] = encoder_df_error_code.fit_transform(df['error_code'])
df['error_description'] = encoder_df_error_description.fit_transform(df['error_description'])
df['cat_name'] = encoder_df_cat_name.fit_transform(df['cat_name'])



**This print gives the results of the encoding**

In [171]:
print(df.max())

testName               test_verification
reportId               MNDORADOVAL-31553
timestamp            2022-10-10 02:26:24
error_description                     17
error_code                            21
cat_name                              75
dtype: object


**Divide in two datasets (one of "training" and one of testing), (training : before today, testing : today)**

In [172]:
ts =  pd.Timestamp(year = 2022, month = 10, day = 9, hour =0 , second = 0)
df_past = df.loc[lambda df: df['timestamp'] < ts]
df = df.drop(df[df.timestamp < ts].index)

**Deleting time from timestamp (only date)**

In [173]:
df['timestamp'] = df['timestamp'].dt.date
df_past['timestamp'] = df_past['timestamp'].dt.date


**Calculate the similarity matrix**

In [174]:
similarity = np.empty((df_past.shape[0],df.shape[0]))
for i in range (0,df_past.shape[0],1):
  for j in range (0, df.shape[0],1):  
    past_vector = np.array([df_past.iloc[i][3],df_past.iloc[i][4],df_past.iloc[i][5]])
    today_vector = np.array([df.iloc[j][3],df.iloc[j][4],df.iloc[j][5]])
    #cosin similarity
    similarity[i,j] = np.dot(past_vector,today_vector)/(norm(past_vector)*norm(today_vector))

**Decoding the features**

In [175]:
df['error_code'] = encoder_df_error_code.inverse_transform(df['error_code'])
df['error_description'] = encoder_df_error_description.inverse_transform(df['error_description'])
df['cat_name'] = encoder_df_cat_name.inverse_transform(df['cat_name'])

**Saving the test set**

In [176]:
df.to_excel('Matches.xlsx', sheet_name='Sightings Tested')

**Reporting the results obtained into the same file**

In [177]:
wb = load_workbook('Matches.xlsx')
ws1 = wb['Sightings Tested']
for i in range(0,similarity.shape[1],1):
  best_match_array = np.where(similarity[:,i] == np.amax(similarity[:,i]))
  best_match = best_match_array[0]
  for j in range(0,len(best_match),1):
     ws1.cell(i+2,j+8).value = df_past.iloc[best_match[j]][1]
wb.save('Matches.xlsx')